### Preparation of configuration files for docking with Gnina

Since Gnina does not have a Python API, we chose to prepare the configuration files required to run Gnina in the `dock/config` directory in advance.  
The docking parameters are consistent with those of AutoDock-Vina, with the docking box center set to the original ligand center or our predefined center  (so please ensure that the center files `center.json` are prepared in `5HT2A-AF` and `BRD4-holo`), with a size of 20 Å.

In [ ]:
import glob
import json
from rdkit import Chem
from pathlib import Path

config_path = Path('../dock/config/')
config_path.mkdir(exist_ok=True)

def sdf2centroid(sdf_file):
    supp = Chem.SDMolSupplier(sdf_file, sanitize=False)
    lig_xyz = supp[0].GetConformer().GetPositions()
    centroid_x = lig_xyz[:,0].mean()
    centroid_y = lig_xyz[:,1].mean()
    centroid_z = lig_xyz[:,2].mean()
    return centroid_x, centroid_y, centroid_z

In [ ]:
for target in Path('./').glob("*/"):
    try:
        x, y, z = sdf2centroid(next(target.glob('*ligand*.sdf')))
    except:
        center_li = json.loads((target/"center.json").read_text())["center"]
        x, y, z = center_li[0], center_li[1], center_li[-1]
    
    conf = f"""center_x = {x}
center_y = {y}
center_z = {z}
size_x = 20
size_y = 20
size_z = 20"""
    with open((config_path/f'{target}.txt'), 'w') as fi:
        fi.write(conf)